# SmartApply Matching Demo
Portfolio showcase of job-resume matching algorithm using synthetic data.

This notebook demonstrates the core matching logic without exposing proprietary algorithms.

In [ ]:
import json
import sys
sys.path.append('../')

from app.services.translator import TranslatorService
from app.services.observability import ObservabilityService

# Initialize services (demo mode)
translator = TranslatorService()
observability = ObservabilityService("demo")

print("✅ Services initialized for demo")

In [ ]:
# Load synthetic data
with open('data/synthetic_jobs.json', 'r') as f:
    jobs = json.load(f)

with open('data/synthetic_resume.json', 'r') as f:
    resume = json.load(f)

print(f"📊 Loaded {len(jobs)} jobs and 1 resume for matching")
print(f"Resume skills: {resume['skills']}")

In [ ]:
def calculate_match_score(job, resume_skills, translator):
    """
    Demo matching algorithm using simple overlap calculation.
    Production uses sophisticated semantic similarity and ML models.
    """
    
    # Extract job requirements
    job_requirements = job.get('requirements', [])
    
    # Translate terms using demo service
    translated_requirements = translator.translate_terms(job_requirements, job['company_name'])
    
    # Calculate overlap (demo logic)
    resume_skills_lower = [skill.lower() for skill in resume_skills]
    translated_lower = [req.lower() for req in translated_requirements]
    
    # Find matches
    matches = []
    for req in translated_lower:
        for skill in resume_skills_lower:
            if req in skill or skill in req:
                matches.append((req, skill))
    
    # Calculate score
    if not job_requirements:
        return 0.0, matches, translated_requirements
    
    score = len(set([m[0] for m in matches])) / len(job_requirements)
    return min(score, 1.0), matches, translated_requirements

print("🔧 Match scoring function defined")

In [ ]:
# Run matching analysis
results = []

with observability.timer("job_matching_batch"):
    for job in jobs:
        score, matches, translated_reqs = calculate_match_score(job, resume['skills'], translator)
        
        result = {
            'job_id': job['id'],
            'company': job['company_name'],
            'role': job['role_title'],
            'match_score': round(score * 100, 1),
            'matches_found': len(matches),
            'key_matches': matches[:3],  # Top 3 matches
            'translated_requirements': translated_reqs,
            'salary_range': job.get('salary_range', 'Not specified')
        }
        results.append(result)
        
        # Log metrics
        observability.record_metric("counter", "jobs_analyzed", 1)
        observability.record_metric("histogram", "match_score", score * 100)

print(f"✅ Analyzed {len(results)} job matches")

In [ ]:
# Display results sorted by match score
results_sorted = sorted(results, key=lambda x: x['match_score'], reverse=True)

print("🎯 TOP JOB MATCHES\n" + "=" * 50)

for i, result in enumerate(results_sorted, 1):
    print(f"\n{i}. {result['role']} at {result['company']}")
    print(f"   Match Score: {result['match_score']}%")
    print(f"   Salary: {result['salary_range']}")
    print(f"   Key Matches: {result['matches_found']} found")
    
    if result['key_matches']:
        print("   Top Matches:")
        for req, skill in result['key_matches']:
            print(f"     • '{req}' ↔ '{skill}'")
    
    print(f"   Job ID: {result['job_id']}")

In [ ]:
# Demonstrate vocabulary translation
print("🔄 VOCABULARY TRANSLATION DEMO\n" + "=" * 40)

sample_terms = ["api", "microservices", "observability", "platform"]
companies = ["PaymentTech", "CloudInfra Systems"]

for company in companies:
    print(f"\n{company}:")
    translations = translator.batch_translate_with_confidence(sample_terms, company)
    
    for translation in translations:
        print(f"  '{translation['original']}' → '{translation['translated']}'")
        print(f"    Confidence: {translation['confidence']:.2f} | Source: {translation['source']}")

In [ ]:
# Show system observability metrics
print("📊 SYSTEM METRICS\n" + "=" * 30)

health = observability.get_health_metrics()
print(f"System Health: {health['health']}")
print(f"Recent API Calls: {health['metrics']['recent_api_calls']}")
print(f"Average Response Time: {health['metrics']['avg_response_time_ms']:.1f}ms")
print(f"Error Rate: {health['metrics']['error_rate']:.2%}")

print("\nFull metrics export:")
print(observability.export_metrics())

In [ ]:
# Demo human-in-the-loop analysis structure
print("👤 HUMAN-IN-THE-LOOP DEMO\n" + "=" * 35)

# Take the top match for demonstration
top_match = results_sorted[0]
print(f"Analyzing: {top_match['role']} at {top_match['company']}")
print(f"Algorithm Score: {top_match['match_score']}%")

# Demo human analysis structure
human_analysis = {
    "job_posting_id": top_match['job_id'],
    "user_id": "demo_user_001",
    "analyst_type": "human",
    "overall_fit_score": 85,  # Human override of algorithm
    "fit_reasoning": "Strong platform experience aligns well with role requirements. Previous billing platform work is highly relevant.",
    "key_matches": {
        "platform": "6+ years platform product management",
        "developer_tools": "Built internal developer tooling",
        "cross_functional": "Led cross-functional alignment initiatives"
    },
    "vocabulary_gaps": {
        "SDK": "Need to emphasize API and integration work",
        "community": "Should highlight developer community engagement"
    },
    "optimization_strategy": "Emphasize developer-facing platform work, highlight API design experience",
    "resume_version_recommended": "platform_focused_v2",
    "confidence_level": 8,
    "estimated_application_priority": "high"
}

print("\nHuman Analysis Structure:")
print(json.dumps(human_analysis, indent=2))

print("\n💡 In production, this analysis would be stored via:")
print("POST /human/role-analysis with full validation and audit trail")

## Summary

This demo showcases:

1. **Job-Resume Matching**: Simple overlap algorithm (production uses ML)
2. **Vocabulary Translation**: Company-specific term mapping
3. **Observability**: Metrics collection and health monitoring  
4. **Human-in-the-Loop**: Structured analysis format for human input

**Portfolio Note**: This demonstrates architecture and data flow patterns. Production includes sophisticated semantic similarity models, proprietary vocabulary engines, and ML-powered matching algorithms that are not included in this public showcase.